<a href="https://colab.research.google.com/github/Joel5120/Colab/blob/main/Another_copy_of_Campus_Place_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### COPY THE FOLLOWING IN CODE CELL 1

##### Campus Placement Prediction Model, The column details are as follows

# GBT Classifier is used in this project

# Total Students : 25,000
# Total number of columns   : 12


###### Column Attributes

# StudentID
# CGPA
# Internships	Projects		- Marks
# Workshops/Certifications	- Marks
# AptitudeTestScore			- Marks
# SoftSkillsRating			- Marks
# ExtracurricularActivities	- Yes/No
# PlacementTraining			- Yes/No
# SSC_Marks
# HSC_Marks
# PlacementStatus				- placed/NotPlaced

#########################################

##### The Core Algorithm used is GBT - Gradient Boosted Trees

# GBT Accuracy                 - 97 %    GBT Classifier is used in this project
# Random Forest Accuracy       - 82 %
# Logistic Regression Accuracy - 52 %
########################################

####### Processing Starts Here

# Step 1 : By default COLAB save your notebook in folder "/content/drive/MyDrive/Colab Notebooks/'
# Copy your notebook from "Colab Notebooks" folder to  "/content/drive/MyDrive/
# Stpe 2 : Use the UNIX Command 'mv' Move command as follows

!mv "/content/drive/MyDrive/Colab Notebooks/Campus_Place_Prediction.ipynb" "/content/drive/MyDrive"

# Import relevant packages to display buttone events

from IPython.display import display, HTML,Javascript
import ipywidgets as VBox
import ipywidgets as widgets
from IPython.display import clear_output

# Define a function to scroll your code cell directly to output grid, other wise you will
# have to manually

def set_focus_cell1():
    display(Javascript("""
        document.getElementById('output-area').scrollIntoView({behavior: 'smooth'});
    """))

# Call set_focus_cell1 to scroll down automatically this cell

set_focus_cell1()

# To get the Processing time of a particular operation import
# the following

import datetime
import time

# Install pyspark, pandas and other ML Packages, 'pip' stands for 'python install packages'

!pip install pyspark
import pyspark
import pandas
from pyspark.ml.classification import LogisticRegression
# Accuracy Evluators
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.feature import VectorAssembler, StringIndexer, IndexToString
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, isnull, regexp_replace, trim, when
import numpy as np # used for array manipulation
import matplotlib.pyplot as plt # used for graph plotting

from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel

from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import GBTClassificationModel

# File Upload package import
from google.colab import files

# For writing single input as csv file
import csv

# To plot graphs import the followng packages

# Seaborn, a statistical data visualization library in Python, is commonly used
# in machine learning for various purposes.

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Co_relation package import

from pyspark.ml.stat import Correlation

# Create an instance of pyspark as follows:-


my_spark = SparkSession.builder.getOrCreate()

mv: cannot stat '/content/drive/MyDrive/Colab Notebooks/Campus_Place_Prediction.ipynb': No such file or directory


<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

def set_focus_cell2():
    display(Javascript("""
        document.getElementById('output-area').scrollIntoView({behavior: 'smooth'});
    """))

# Call set_focus to scroll down automatically this cell

set_focus_cell2()

# Clear output aread Fn

def clear_result_grid():
    clear_output(wait=True)

def load_data(event):
  print()
# Record the start time (current_time) before the process starts
  current_time = datetime.datetime.now()
  print("Current Time :", current_time.strftime("%I:%M:%S %p"))  # Format the datetime
  display(HTML("<h3 style='color:orange; text-align:left;'>Loading Data, this may take few seconds...wait !!!</h3>"))

  global df_from_csv
# Read and Load your data

  df_from_csv = my_spark.read \
                  .format("csv") \
                  .option("header", "True") \
                  .option("inferSchema", "True") \
                  .load("/content/drive/MyDrive/Campus_Place_Syn.csv")

# Cross Verification

  print("Schema & Summary of df_from_CSV")
  df_from_csv.printSchema()

# Schema & Summary of df_from_CSV
# root
#  |-- StudentID: integer (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: integer (nullable = true)
#  |-- Projects: integer (nullable = true)
#  |-- Workshops_Certifications: integer (nullable = true)
#  |-- AptitudeTestScore: integer (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: integer (nullable = true)
#  |-- HSC_Marks: integer (nullable = true)
#  |-- PlacementStatus: string (nullable = true)

  print("Dispaly few data in df_from_CSV")
  df_from_csv.show(5, False)

# Dispaly few data in df_from_CSV
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+
# |StudentID|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|PlacementStatus|
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+
# |445      |6.5 |1          |2       |0                       |74               |3.5             |No                       |No               |59       |88       |Placed         |
# |5138     |6.6 |1          |1       |0                       |69               |3.6             |No                       |No               |59       |72       |NotPlaced      |
# |125      |6.8 |1          |3       |1                       |77               |4.3             |No                       |No               |71       |62       |NotPlaced      |
# |2055     |7.5 |0          |1       |0                       |78               |4.2             |No                       |No               |55       |61       |NotPlaced      |
# |7607     |6.5 |0          |2       |1                       |82               |3.8             |No                       |Yes              |65       |69       |Placed         |
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+


# Record the end time after the process has finished

  end_time = datetime.datetime.now()
  print("End Time H:M:S :", end_time.strftime("%I:%M:%S %p"))  # Format the datetime

# End Time H:M:S : 10:17:33 AM

# Calculate the time difference
  time_difference = end_time - current_time

# Print the time difference
  formatted_time_difference = str(time_difference).split('.')[0]  # Remove microseconds
  print("Loading Time :", formatted_time_difference)

# Loading Time : 0:00:13 Seconds

  tot_count = df_from_csv.count()
  print("Total number of Students = ",tot_count)

# Total number of Students =  25000

  print()
  display(HTML("<h3 style='color:magenta; text-align:left;'>All your Data loaded Successfully...</h3>"))
  main_function()

def EDA_start(event):
  print()
  clear_result_grid()
  set_focus_cell2()

  display(HTML("<h3 style='color:orange; text-align:left;'>EDA Started, Please wait !!!</h3>"))

# Display overall statistics of df_from_csv data

  print("Statistics of df_from_csv Dataframe...")
  df_from_csv.describe().show()

# Statistics of df_from_csv Dataframe...
# +-------+-----------------+------------------+------------------+------------------+------------------------+------------------+-------------------+-------------------------+-----------------+------------------+-----------------+---------------+
# |summary|        StudentID|              CGPA|       Internships|          Projects|Workshops_Certifications| AptitudeTestScore|   SoftSkillsRating|ExtracurricularActivities|PlacementTraining|         SSC_Marks|        HSC_Marks|PlacementStatus|
# +-------+-----------------+------------------+------------------+------------------+------------------------+------------------+-------------------+-------------------------+-----------------+------------------+-----------------+---------------+
# |  count|            25000|             25000|             25000|             25000|                   25000|             25000|              25000|                    25000|            25000|             25000|            25000|          25000|
# |   mean|        5494.5476| 7.744368000000107|           1.10312|           2.17316|                  1.0034|          80.69516| 4.3711680000003295|                     NULL|             NULL|           70.7934|         76.06488|           NULL|
# | stddev|2970.788566128945|0.6570083309775292|0.7017876928520707|0.8547308564650544|      0.9149764626902107|7.8604417201682555|0.39828515038655565|                     NULL|             NULL|10.253555571413948|8.747080143798945|           NULL|
# |    min|                1|               6.5|                 0|                 0|                       0|                60|                3.0|                       No|               No|                55|               57|      NotPlaced|
# |    max|             9995|               9.1|                 2|                 3|                       3|                90|                4.8|                      Yes|              Yes|                90|               88|         Placed|
# +-------+-----------------+------------------+------------------+------------------+------------------------+------------------+-------------------+-------------------------+-----------------+------------------+-----------------+---------------+


# Display the total count of distinct rows

  tot_count = df_from_csv.count()
  distinct_count = df_from_csv .distinct().count()

  print("Total count of Duplicate Rows = ", tot_count - distinct_count)

# Total count of Duplicate Rows =  0

  result_counts = df_from_csv.groupBy("PlacementStatus").count()

  print("Total Count of Placed/NotPlaced")
  result_counts.show()

# Total Count of Placed/NotPlaced
# +---------------+-----+
# |PlacementStatus|count|
# +---------------+-----+
# |      NotPlaced|12111|
# |         Placed|12889|
# +---------------+-----+

  print()
  display(HTML("<h3 style='color:magenta; text-align:left;'>EDA Operations finished...</h3>"))
# Return control to main_function
  main_function()

def pre_process_data(event):
  print()
  print()
  clear_result_grid()
  global df_final
  display(HTML("<h3 style='color:orange; text-align:left;'>Pre-Processing of Data Started, Please wait !!!</h3>"))

# Remove 'null' values from the above columns

  columns_to_clean = ["StudentID","CGPA","Internships","Projects","Workshops_Certifications",
                      "AptitudeTestScore","SoftSkillsRating","ExtracurricularActivities",
                      "PlacementTraining","SSC_Marks","HSC_Marks","PlacementStatus"
                     ]

  df_final = df_from_csv.na.drop(subset=columns_to_clean)

######## Important Note : How to interpret Standard Deviation with mean

# Spread around the Mean:

# The standard deviation provides information about how spread out the values in a dataset are.
# A low standard deviation indicates that the values are close to the mean, suggesting less variability.
# A high standard deviation suggests that the values are more spread out from the mean,
# indicating greater variability.
########

# Cross Verify

  print("Statistics of 'df_final'")
  df_final.describe().show()

# Statistics of 'df_final'
# +-------+-----------------+------------------+------------------+------------------+------------------------+------------------+-------------------+-------------------------+-----------------+------------------+-----------------+---------------+
# |summary|        StudentID|              CGPA|       Internships|          Projects|Workshops_Certifications| AptitudeTestScore|   SoftSkillsRating|ExtracurricularActivities|PlacementTraining|         SSC_Marks|        HSC_Marks|PlacementStatus|
# +-------+-----------------+------------------+------------------+------------------+------------------------+------------------+-------------------+-------------------------+-----------------+------------------+-----------------+---------------+
# |  count|            25000|             25000|             25000|             25000|                   25000|             25000|              25000|                    25000|            25000|             25000|            25000|          25000|
# |   mean|        5494.5476| 7.744368000000107|           1.10312|           2.17316|                  1.0034|          80.69516| 4.3711680000003295|                     NULL|             NULL|           70.7934|         76.06488|           NULL|
# | stddev|2970.788566128945|0.6570083309775292|0.7017876928520707|0.8547308564650544|      0.9149764626902107|7.8604417201682555|0.39828515038655565|                     NULL|             NULL|10.253555571413948|8.747080143798945|           NULL|
# |    min|                1|               6.5|                 0|                 0|                       0|                60|                3.0|                       No|               No|                55|               57|      NotPlaced|
# |    max|             9995|               9.1|                 2|                 3|                       3|                90|                4.8|                      Yes|              Yes|                90|               88|         Placed|
# +-------+-----------------+------------------+------------------+------------------+------------------------+------------------+-------------------+-------------------------+-----------------+------------------+-----------------+---------------+

  display(HTML("<h3 style='color:magenta; text-align:left;'>Pre-Processing Finished...</h3>"))
  main_function()

def transformation(event):
  print()
  clear_result_grid()
  set_focus_cell2()
  global label_df

  display(HTML("<h3 style='color:orange; text-align:left;'> \
    Transformation Started may take less than 1 Minute, Please wait !!!</h3>"))
  print()

# in the df_final Dataframe the following column's data types are 'integer'
# Convert those columns to Decimal vectors as follows

#  |-- Internships: integer (nullable = true)
#  |-- Projects: integer (nullable = true)
#  |-- Workshops_Certifications: integer (nullable = true)
#  |-- AptitudeTestScore: integer (nullable = true)
#  |-- SSC_Marks: integer (nullable = true)
#  |-- HSC_Marks: integer (nullable = true)


#
  df_data_type_converted = df_final.withColumn("Internships",
                                    col("Internships").cast("double"))
  df_data_type_converted = df_data_type_converted.withColumn("Projects",
                                    col("Projects").cast("double"))
  df_data_type_converted = df_data_type_converted.withColumn("Workshops_Certifications",
                                    col("Workshops_Certifications").cast("double"))
  df_data_type_converted = df_data_type_converted.withColumn("AptitudeTestScore",
                                    col("AptitudeTestScore").cast("double"))
  df_data_type_converted = df_data_type_converted.withColumn("SSC_Marks",
                                    col("SSC_Marks").cast("double"))
  df_data_type_converted = df_data_type_converted.withColumn("HSC_Marks",
                                    col("HSC_Marks").cast("double"))

# Cross Verify

  print("Display Schema of df_data_type_converted")
  df_data_type_converted.printSchema()
# Display Schema of df_data_type_converted
# root
#  |-- StudentID: integer (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: double (nullable = true)
#  |-- Projects: double (nullable = true)
#  |-- Workshops_Certifications: double (nullable = true)
#  |-- AptitudeTestScore: double (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: double (nullable = true)
#  |-- HSC_Marks: double (nullable = true)
#  |-- PlacementStatus: string (nullable = true)

  print("Display few Data in df_data_type_converted")
  df_data_type_converted.show(5, False)
# Display few Data in df_data_type_converted
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+
# |StudentID|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|PlacementStatus|
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+
# |445      |6.5 |1.0        |2.0     |0.0                     |74.0             |3.5             |No                       |No               |59.0     |88.0     |Placed         |
# |5138     |6.6 |1.0        |1.0     |0.0                     |69.0             |3.6             |No                       |No               |59.0     |72.0     |NotPlaced      |
# |125      |6.8 |1.0        |3.0     |1.0                     |77.0             |4.3             |No                       |No               |71.0     |62.0     |NotPlaced      |
# |2055     |7.5 |0.0        |1.0     |0.0                     |78.0             |4.2             |No                       |No               |55.0     |61.0     |NotPlaced      |
# |7607     |6.5 |0.0        |2.0     |1.0                     |82.0             |3.8             |No                       |Yes              |65.0     |69.0     |Placed         |
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+

# Convert ExtracurricularActivities,PlacementTraining to vectors as follows:-

#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)

#  |-- PlacementStatus: string (nullable = true) Label column

# Extracurr- Yes/No, PlacementTraining Yes/No

  df_Extracurricular_converted = df_data_type_converted.withColumn("Extracurricular_indexed",
                   when(df_data_type_converted.ExtracurricularActivities == "Yes", 0.0)
                  .when(df_data_type_converted.ExtracurricularActivities == "No", 1.0)
                  .otherwise(2.0)
                          )

  df_PlaceTraining_converted = df_Extracurricular_converted.withColumn("PlacementTraining_indexed",
                   when(df_data_type_converted.PlacementTraining == "Yes", 0.0)
                  .when(df_data_type_converted.PlacementTraining == "No", 1.0)
                  .otherwise(2.0)
                          )
# Cross Verify

  print("Display few Data in df_PlaceTraining_converted")
  df_PlaceTraining_converted.show(5, False)
# Display few Data in df_PlaceTraining_converted
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+
# |StudentID|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|PlacementStatus|Extracurricular_indexed|PlacementTraining_indexed|
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+
# |445      |6.5 |1.0        |2.0     |0.0                     |74.0             |3.5             |No                       |No               |59.0     |88.0     |Placed         |1.0                    |1.0                      |
# |5138     |6.6 |1.0        |1.0     |0.0                     |69.0             |3.6             |No                       |No               |59.0     |72.0     |NotPlaced      |1.0                    |1.0                      |
# |125      |6.8 |1.0        |3.0     |1.0                     |77.0             |4.3             |No                       |No               |71.0     |62.0     |NotPlaced      |1.0                    |1.0                      |
# |2055     |7.5 |0.0        |1.0     |0.0                     |78.0             |4.2             |No                       |No               |55.0     |61.0     |NotPlaced      |1.0                    |1.0                      |
# |7607     |6.5 |0.0        |2.0     |1.0                     |82.0             |3.8             |No                       |Yes              |65.0     |69.0     |Placed         |1.0                    |0.0                      |
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+

# Select your feature columns for training & testing continue from here

  feature_cols = ["CGPA","Internships","Projects","Workshops_Certifications",
                      "AptitudeTestScore","SoftSkillsRating","Extracurricular_indexed",
                      "PlacementTraining_indexed","SSC_Marks","HSC_Marks"
                 ]
  output_col = "features"


# Vectorize the above feature_cols and add it as a list in df_final_transformed

  assembler = VectorAssembler(inputCols=feature_cols, outputCol=output_col)

  df_final_vectors = assembler.transform(df_PlaceTraining_converted)

# Cross verify schema of df_final_vectors, a new column features will be added

  print("Schema of df_final_vectors")
  df_final_vectors.printSchema()

# Schema of df_final_vectors
# root
#  |-- StudentID: integer (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: double (nullable = true)
#  |-- Projects: double (nullable = true)
#  |-- Workshops_Certifications: double (nullable = true)
#  |-- AptitudeTestScore: double (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: double (nullable = true)
#  |-- HSC_Marks: double (nullable = true)
#  |-- PlacementStatus: string (nullable = true)
#  |-- Extracurricular_indexed: double (nullable = false)
#  |-- PlacementTraining_indexed: double (nullable = false)
#  |-- features: vector (nullable = true)         New Vector col added

 # Cross Verify features data

  print("Display few Data in df_final_vectors")
  df_final_vectors.show(5, False)

# Display few Data in df_final_vectors
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+
# |StudentID|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|PlacementStatus|Extracurricular_indexed|PlacementTraining_indexed|features                                    |
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+
# |445      |6.5 |1.0        |2.0     |0.0                     |74.0             |3.5             |No                       |No               |59.0     |88.0     |Placed         |1.0                    |1.0                      |[6.5,1.0,2.0,0.0,74.0,3.5,1.0,1.0,59.0,88.0]|
# |5138     |6.6 |1.0        |1.0     |0.0                     |69.0             |3.6             |No                       |No               |59.0     |72.0     |NotPlaced      |1.0                    |1.0                      |[6.6,1.0,1.0,0.0,69.0,3.6,1.0,1.0,59.0,72.0]|
# |125      |6.8 |1.0        |3.0     |1.0                     |77.0             |4.3             |No                       |No               |71.0     |62.0     |NotPlaced      |1.0                    |1.0                      |[6.8,1.0,3.0,1.0,77.0,4.3,1.0,1.0,71.0,62.0]|
# |2055     |7.5 |0.0        |1.0     |0.0                     |78.0             |4.2             |No                       |No               |55.0     |61.0     |NotPlaced      |1.0                    |1.0                      |[7.5,0.0,1.0,0.0,78.0,4.2,1.0,1.0,55.0,61.0]|
# |7607     |6.5 |0.0        |2.0     |1.0                     |82.0             |3.8             |No                       |Yes              |65.0     |69.0     |Placed         |1.0                    |0.0                      |[6.5,0.0,2.0,1.0,82.0,3.8,1.0,0.0,65.0,69.0]|
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+

# Create 'label' column as follows:-
# The label column or the Diagnosis column in this project is 'PlacementStatus'
# Convert this to decimal vector using String Indexer

  PlacementStatus_indexer = StringIndexer(
    inputCol="PlacementStatus",
    outputCol="label",
    stringOrderType="alphabetAsc"  # You can set the desired sort order here
    ).setHandleInvalid("skip")

# Since String Indexer is used you should use 'fit' and 'transform'

  label_df = PlacementStatus_indexer.fit(df_final_vectors).transform(df_final_vectors)

# Cross Verify, whether a label column is added

  print("Schema of label_df")
  label_df.printSchema()
# Schema of label_df
# root
#  |-- StudentID: integer (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: double (nullable = true)
#  |-- Projects: double (nullable = true)
#  |-- Workshops_Certifications: double (nullable = true)
#  |-- AptitudeTestScore: double (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: double (nullable = true)
#  |-- HSC_Marks: double (nullable = true)
#  |-- PlacementStatus: string (nullable = true)
#  |-- Extracurricular_indexed: double (nullable = false)
#  |-- PlacementTraining_indexed: double (nullable = false)
#  |-- features: vector (nullable = true)
#  |-- label: double (nullable = false)       New col added

  print("Display few Data in label_df")
  label_df.show(5, False)
# Display few Data in label_df
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+
# |StudentID|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|PlacementStatus|Extracurricular_indexed|PlacementTraining_indexed|features                                    |label|
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+
# |445      |6.5 |1.0        |2.0     |0.0                     |74.0             |3.5             |No                       |No               |59.0     |88.0     |Placed         |1.0                    |1.0                      |[6.5,1.0,2.0,0.0,74.0,3.5,1.0,1.0,59.0,88.0]|1.0  |
# |5138     |6.6 |1.0        |1.0     |0.0                     |69.0             |3.6             |No                       |No               |59.0     |72.0     |NotPlaced      |1.0                    |1.0                      |[6.6,1.0,1.0,0.0,69.0,3.6,1.0,1.0,59.0,72.0]|0.0  |
# |125      |6.8 |1.0        |3.0     |1.0                     |77.0             |4.3             |No                       |No               |71.0     |62.0     |NotPlaced      |1.0                    |1.0                      |[6.8,1.0,3.0,1.0,77.0,4.3,1.0,1.0,71.0,62.0]|0.0  |
# |2055     |7.5 |0.0        |1.0     |0.0                     |78.0             |4.2             |No                       |No               |55.0     |61.0     |NotPlaced      |1.0                    |1.0                      |[7.5,0.0,1.0,0.0,78.0,4.2,1.0,1.0,55.0,61.0]|0.0  |
# |7607     |6.5 |0.0        |2.0     |1.0                     |82.0             |3.8             |No                       |Yes              |65.0     |69.0     |Placed         |1.0                    |0.0                      |[6.5,0.0,2.0,1.0,82.0,3.8,1.0,0.0,65.0,69.0]|1.0  |
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+

# Co_relation example of feature columns, How to interpret results of
# Pearson coefficient

# pearson coeff  = 0.0   Weak linearlity, or correlation is weak
# pearson coeff  = 1.0   High linearlity, or correlation is Good
# pearson coeff  = -0.0, or negative values   No linearlity, or No correlation


# # Compute the correlation matrix using Correlation module, this method use
# # PEARSON  correlation coefficient

# Using DataFrame API to compute the correlation matrix CGPA,Internships

  correlation_matrix_coeff = label_df.stat.corr("CGPA","Internships", method='pearson')

  print("Correlation Matrix using CGPA,Internships= ", correlation_matrix_coeff)

# Correlation Matrix using CGPA,Internships=  0.2937236121227813

  correlation_matrix_coeff = label_df.stat.corr("CGPA","Projects", method='pearson')

  correlation_matrix_coeff = label_df.stat.corr("CGPA","Workshops_Certifications", method='pearson')

  print("Correlation Matrix using CGPA,Workshops_Certifications= ", correlation_matrix_coeff)

# Correlation Matrix using CGPA,Workshops_Certifications=  0.3237425822971243

  correlation_matrix_coeff = label_df.stat.corr("CGPA","AptitudeTestScore", method='pearson')

  print("Correlation Matrix using CGPA,AptitudeTestScore= ", correlation_matrix_coeff)

# Correlation Matrix using CGPA,AptitudeTestScore=  0.4467826454514333

  correlation_matrix_coeff = label_df.stat.corr("CGPA","SoftSkillsRating", method='pearson')

  print("Correlation Matrix using CGPA,SoftSkillsRating= ", correlation_matrix_coeff)

# Correlation Matrix using CGPA,SoftSkillsRating=  0.3896262497244297

  correlation_matrix_coeff = label_df.stat.corr("CGPA","SSC_Marks", method='pearson')

  print("Correlation Matrix using CGPA,SSC_Marks= ", correlation_matrix_coeff)

# Correlation Matrix using CGPA,SSC_Marks=  0.4006082835525229

# ######## Similarly combine other columns taking 2 columns at a time

# # it's not exactly 1, the value you have (0.9389) indicates an extremely strong positive
# # correlation between the two variables. It's common to interpret correlation coefficients
# # close to 1 as indicative of a very strong linear relationship.
# # Negative values has got very weak linear relationship.

# ##########

  display(HTML("<h3 style='color:magenta; text-align:left;'>Transformation Finished...</h3>"))

  main_function()

def build_model_save(event):
  print()

  display(HTML("<h3 style='color:magenta; text-align:left;'> \
    Model Building Started Please Wait...will take 1 to 2 minutes..</h3>"))

# Record the start time (current_time) before the process starts
  current_time = datetime.datetime.now()
  print("Current Time :", current_time.strftime("%I:%M:%S %p"))  # Format the datetime

############# Prepare Training and Testing Data

# Important : The Split label_df, 70 % for Training and 30 % for Testing
# not suited for this project because the model not giving correct results
# when actual end user data was given. This happens because of 'OVERFITTING'.
# So change split ratio to 65 % - 35 % to get correct results
###############

  train_data_df, test_data_df = label_df.randomSplit([0.65, 0.35], seed=5043)

# Count total number of rows in train_data_df, test_data_df

  tot_train = train_data_df.count()
  print("Total Rows in Training Dataset..=", tot_train)

# Total Rows in Training Dataset..= 17355

  tot_test = test_data_df.count()
  print("Total Rows in Test Dataset..=", tot_test)

# Total Rows in Test Dataset..= 7645

# Create a Logistic Regression model and set the labelCol to 'label'

  lr = LogisticRegression(maxIter=10, regParam=0.3,
                          elasticNetParam=0.8, featuresCol="features",
                          labelCol='label')

# Fit the model to your data

  lr_model = lr.fit(train_data_df)

# Predict using LR, apply test data on top of lr_model

  prediction_lr = lr_model.transform(test_data_df)

####### Evaluate your accuracy of model using Binary classificiation
# ROC - Receiver Operating Curve or AUC

  evaluator_lr_auc = BinaryClassificationEvaluator(labelCol="label",
                                                   rawPredictionCol="prediction",
                                                   metricName="areaUnderROC")

####### Metrics Explanation using MultiClass Classification
# For multi-class classification in PySpark, you can use metrics such as:

# accuracy: To calculate the accuracy of the model's predictions.
# weightedPrecision: To calculate the weighted precision.
# weightedRecall: To calculate the weighted recall.
# weightedFMeasure: To calculate the weighted F1 score.
# weightedFalsePositiveRate: To calculate the weighted false positive rate.

# Accuracy is a straightforward and widely used metric that measures the overall correctness
# of a model's predictions.

# Mathematically, it is defined as:

# Accuracy = (Number of Correct Predictions) / (Total Number of Predictions)


  # evaluator_lr_accuracy = MulticlassClassificationEvaluator(labelCol="label",
  #                                                  predictionCol="prediction",
  #                                                  metricName="accuracy")

  evaluator_lr_auc_accuracy = evaluator_lr_auc.evaluate(prediction_lr)
  lr_accuracy_rounded = round(evaluator_lr_auc_accuracy * 100, 2)

  print()
  print("Accuracy of Prediction using LR with Different Metrics....")
  print("==========================================================")
  print("Accuracy of LR using AUC = ", lr_accuracy_rounded)

# Accuracy of Prediction using LR with Different Metrics....
# ==========================================================
# Accuracy of LR using AUC =  50.0

###########


####### Evaluate your accuracy of model using 'weightedRecall' or F1 score

# Weighted recall is a more robust metric when dealing with imbalanced datasets, as it
# gives each class a fair contribution to the overall evaluation.

# The formula for weighted recall is:

# Weighted Recall = Σ (Recall for each class * Proportion of that class in the dataset)

#########

  # evaluator_lr_recall = MulticlassClassificationEvaluator(labelCol="label",
  #                                                  predictionCol="prediction",
  #                                                  metricName="weightedRecall")

  # evaluator_lr_recall_accuracy = evaluator_lr_recall.evaluate(prediction_lr)
  # lr_recall_rounded = round(evaluator_lr_recall_accuracy * 100, 2)

  # print("Accuracy of LR using FMeasure = ", lr_recall_rounded)

  print()
  display(HTML("<h3 style='color:green; text-align:left;'>Processing Random Forest Please Wait !!!</h3>"))


########### Logistic Regression finished

##### Create a Random Forest Model and set the labelCol to 'label'

# The 'setImpurity' criteria in RF can be 'gini', 'variance', 'entropy'

# 'entropy' is used to eliminate 'over fitting' of train data

# There are several different 'feature subset' strategies available, including:
# you should try different feature subsets to get max accuracy

# "all": Considers all features at each split.
# "random": Considers a random subset of features at each split.
# "sqrt": Considers a random subset of features that is equal to the square root of the
# total number of features.
# "log2": Considers a random subset of features that is equal to the log2 of the
# total number of features.

  rf = RandomForestClassifier() \
        .setImpurity("entropy") \
        .setNumTrees(20) \
        .setMaxDepth(20) \
        .setFeatureSubsetStrategy("all") \
        .setSeed(5043) # Used to get fixed values in each run

# Fit the model to your data

  rf_model = rf.fit(train_data_df)

# Predict using RF, apply test data on top of rf_model

  prediction_rf = rf_model.transform(test_data_df)

# Cross Verify

  # print("Display Data in predicition_df...")
  # prediction_rf.show(5, False)

# Assuming your label column is named 'label' and your raw prediction
# column is named 'rawPrediction'

  evaluator_rf_binary = BinaryClassificationEvaluator(labelCol="label",
                                                      rawPredictionCol="rawPrediction",
                                                      metricName="areaUnderROC")

# Calculate and print the area under the ROC curve on the training data

  rf_auc_binary = evaluator_rf_binary.evaluate(prediction_rf)

  rf_auc_rounded = round(rf_auc_binary * 100, 2)

  print("Accuracy Area under ROC :", rf_auc_rounded)

# Accuracy Area under ROC : 82.79

####### Evaluate your accuracy of model using ACCURACY Metric

  # evaluator_rf_accuracy = MulticlassClassificationEvaluator(labelCol="label",
  #                                                  predictionCol="prediction",
  #                                                  metricName="accuracy")


  # evaluator_rf_auc_accuracy = evaluator_rf_accuracy.evaluate(prediction_rf)
  # rf_accuracy_rounded = round(evaluator_rf_auc_accuracy * 100, 2)

  # print()
  # print("Accuracy of Prediction using RF with Different Metrics....")
  # print("==========================================================")
  # print("Accuracy of RF using accuracy metrics = ", rf_accuracy_rounded)

### Evaluate your accuracy of model using 'weightedRecall' or F1 score

  # evaluator_rf_recall = MulticlassClassificationEvaluator(labelCol="label",
  #                                                  predictionCol="prediction",
  #                                                  metricName="weightedRecall")

  # evaluator_rf_recall_accuracy = evaluator_rf_recall.evaluate(prediction_rf)
  # rf_recall_rounded = round(evaluator_rf_recall_accuracy * 100, 2)

  # print("Accuracy of RF using FMeasure = ", rf_recall_rounded)


############ Random Forest ends here

# ########### GBT Starts here

# Create a Gradient Booster Tree and set the labelCol to 'label'

  print()
  display(HTML("<h3 style='color:green; text-align:left;'>Processing GBT Please Wait !!!</h3>"))

  gbt = GBTClassifier(
                      maxDepth = 10,
                      maxIter = 30,  # This corresponds to the number of trees in GBT
                      seed=5043
                      )

# # # # Fit the model to your data

  gbt_model = gbt.fit(train_data_df)

  prediction_gbt = gbt_model.transform(train_data_df)

  print("Display GBT Prediction Results in prediction_gbt")
  prediction_gbt.show(30, False)

# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+------------------------------------------+-----------------------------------------+----------+
# |StudentID|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|PlacementStatus|Extracurricular_indexed|PlacementTraining_indexed|features                                    |label|rawPrediction                             |probability                              |prediction|
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+------------------------------------------+-----------------------------------------+----------+
# |1        |6.9 |0.0        |1.0     |0.0                     |66.0             |4.4             |Yes                      |No               |56.0     |67.0     |NotPlaced      |0.0                    |1.0                      |[6.9,0.0,1.0,0.0,66.0,4.4,0.0,1.0,56.0,67.0]|0.0  |[1.3345658180032591,-1.3345658180032591]  |[0.9351804127850412,0.06481958721495884] |0.0       |
# |4        |6.5 |2.0        |3.0     |3.0                     |89.0             |4.8             |Yes                      |No               |86.0     |84.0     |Placed         |0.0                    |1.0                      |[6.5,2.0,3.0,3.0,89.0,4.8,0.0,1.0,86.0,84.0]|1.0  |[-1.3971560375367806,1.3971560375367806]  |[0.057632315675267436,0.9423676843247326]|1.0       |
# |5        |8.2 |2.0        |1.0     |2.0                     |90.0             |4.8             |Yes                      |Yes              |83.0     |81.0     |Placed         |0.0                    |0.0                      |[8.2,2.0,1.0,2.0,90.0,4.8,0.0,0.0,83.0,81.0]|1.0  |[-1.3888810362957775,1.3888810362957775]  |[0.058537768398425095,0.9414622316015749]|1.0       |
# |7        |6.6 |1.0        |2.0     |1.0                     |78.0             |4.3             |No                       |Yes              |58.0     |70.0     |NotPlaced      |1.0                    |0.0                      |[6.6,1.0,2.0,1.0,78.0,4.3,1.0,0.0,58.0,70.0]|0.0  |[0.8283113200780285,-0.8283113200780285]  |[0.8397841115454632,0.16021588845453683] |0.0       |
# |7        |6.8 |1.0        |3.0     |0.0                     |73.0             |4.3             |No                       |Yes              |59.0     |73.0     |NotPlaced      |1.0                    |0.0                      |[6.8,1.0,3.0,0.0,73.0,4.3,1.0,0.0,59.0,73.0]|0.0  |[1.3798380832136206,-1.3798380832136206]  |[0.9404575028402804,0.0595424971597196]  |0.0       |
# |7        |8.0 |1.0        |3.0     |2.0                     |87.0             |4.6             |Yes                      |No               |72.0     |85.0     |Placed         |0.0                    |1.0                      |[8.0,1.0,3.0,2.0,87.0,4.6,0.0,1.0,72.0,85.0]|1.0  |[-1.0108846638903588,1.0108846638903588]  |[0.11693616251284862,0.8830638374871513] |1.0       |
# |7        |8.2 |2.0        |3.0     |0.0                     |90.0             |4.8             |Yes                      |Yes              |88.0     |84.0     |Placed         |0.0                    |0.0                      |[8.2,2.0,3.0,0.0,90.0,4.8,0.0,0.0,88.0,84.0]|1.0  |[-1.4136980791824783,1.4136980791824783]  |[0.055861568569531714,0.9441384314304683]|1.0       |
# |9        |8.1 |1.0        |2.0     |2.0                     |74.0             |4.2             |Yes                      |Yes              |75.0     |77.0     |NotPlaced      |0.0                    |0.0                      |[8.1,1.0,2.0,2.0,74.0,4.2,0.0,0.0,75.0,77.0]|0.0  |[0.4178411513896231,-0.4178411513896231]  |[0.6975550805710566,0.30244491942894336] |0.0       |
# |10       |7.7 |1.0        |1.0     |0.0                     |73.0             |4.7             |No                       |Yes              |72.0     |72.0     |Placed         |1.0                    |0.0                      |[7.7,1.0,1.0,0.0,73.0,4.7,1.0,0.0,72.0,72.0]|1.0  |[0.18652758250875187,-0.18652758250875187]|[0.5921970072159141,0.40780299278408594] |0.0       |
# |10       |7.8 |0.0        |3.0     |2.0                     |83.0             |4.2             |No                       |Yes              |73.0     |83.0     |NotPlaced      |1.0                    |0.0                      |[7.8,0.0,3.0,2.0,83.0,4.2,1.0,0.0,73.0,83.0]|0.0  |[0.881112788883351,-0.881112788883351]    |[0.8534881790349017,0.1465118209650983]  |0.0       |
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+------------------------------------------+-----------------------------------------+----------+
#

# Evaluate accuracy of gbt

  evaluator_gbt_binary = BinaryClassificationEvaluator(labelCol="label",
                                                       rawPredictionCol="rawPrediction",
                                                       metricName="areaUnderROC")

# Calculate and print the area under the ROC curve on the training data

  auc_gbt = evaluator_gbt_binary.evaluate(prediction_gbt)
  auc_gbt_rounded = round(auc_gbt * 100, 2)
  print("Accuracy Area under ROC :", auc_gbt_rounded)

# Accuracy Area under ROC : 97.08

########## Plot Scatter and Line Graph

# To create a scatter plot extract 'label' and 'prediction' from
# prediction_df Dataframe, then convert it to pandas dataframe

# Create Actual scatter plot using the predicted value and labels

  labels = prediction_gbt.select("label").toPandas()
  predictions = prediction_gbt.select("prediction").toPandas()

 # Create a actual scatter plot

  plt.figure(figsize=(4, 3))
  plt.scatter(predictions, labels)
  plt.xlabel("Predictions")
  plt.ylabel("Labels")
  plt.title("Scatter Plot Actual")
  plt.show()

# Create a line plot

  plt.figure(figsize=(4, 3))
  plt.plot(predictions, labels, marker='.', linestyle='-', color='red')
  plt.xlabel("Predictions")
  plt.ylabel("Labels")
  plt.title("Line Plot")
  plt.show()

# # Draw a Voilin Plot

#   confu_matrix = confusion_matrix(labels, predictions.round())
#   plt.figure(figsize=(4, 3))
#   sns.violinplot(confu_matrix, annot=True, fmt="d", cmap="Dark2")
#   plt.xlabel("Predicted Labels")
#   plt.ylabel("Actual Labels")
#   plt.title("Violin Plot, with Noise")
#   plt.show()

# # Draw a Confusion Matrix
# # cmap or colormap codes are, 'Set1', "Dark2", "Blues", "Accent", "Paired"

#   confu_matrix = confusion_matrix(labels, predictions.round())
#   plt.figure(figsize=(4, 3))
#   sns.heatmap(confu_matrix, annot=True, fmt="d", cmap="Set1")
#   plt.xlabel("Predicted Labels")
#   plt.ylabel("Actual Labels")
#   plt.title("Confusion Matrix Heatmap, with Noise")
#   plt.show()

####### Filter out unmatched predictions and display scatter plot

# Filter out points not on the diagonal, that is remove erroneous label and
# prediction.

  filtered_df = prediction_rf.filter(col("label") == col("prediction"))

  labels = filtered_df.select("label").toPandas()
  predictions = filtered_df.select("prediction").toPandas()

# Create a scatter plot using only label and prediction are equal

  # plt.figure(figsize=(4, 3))
  # plt.scatter(predictions, labels)
  # plt.xlabel("Predictions")
  # plt.ylabel("Labels")

  # plt.title("Scatter Plot, Noise Removed")
  # plt.show()

# Create a LINE plot using only label and prediction are equal

  plt.figure(figsize=(4, 3))
  plt.plot(predictions, labels, marker='.', linestyle='-', color='red')
  plt.xlabel("Predictions")
  plt.ylabel("Labels")
  plt.title("Line Plot, Noise Removed")
  plt.show()


# cmap or colormap codes are, 'Set1', "Dark2", "Blues", "Accent", "Paired"


###### Draw a Confusion Matrix, How to interpret Confusion Matrix

# If you see a dark cell for "Class A" (actual) and "Class B" (predicted) and it's not on
# the diagonal, it means that the model is misclassifying instances of "Class A" as "Class B."

# In a well-performing model, you would generally want to see bright colors (indicating high
# counts) along the diagonal and lighter colors (indicating lower counts) elsewhere,
# suggesting that most instances are correctly classified. Darker colors away from the
# diagonal indicate areas where the model is making more errors.
#######

  confu_matrix = confusion_matrix(labels, predictions.round())
  plt.figure(figsize=(4, 3))
  sns.heatmap(confu_matrix, annot=True, fmt="d", cmap="Dark2")
  plt.xlabel("Predicted Labels")
  plt.ylabel("Actual Labels")
  plt.title("Confusion Matrix Heatmap, Noise Removed")
  plt.show()

# Cross verify, how many columns shows disimilarity

  filtered_df = prediction_rf.filter(col("label") != col("prediction"))
  print("Count of un Matched Predictions")
  un_matched = filtered_df.count()
  print("Un Matched Data...", un_matched)

# Count of Un Matched Data... 1830, with 70-30 split ratio

# In the Confusion matrix the Correct predictions from test data is calculated as follows :-

# Tot Rows in Test Data = 4559
# Diagonal Total from Confusion Matrix = 4538
# False predictions = 4559 - 4524 = 35, Tallied with Un Matched Data....= 21

# Display unmatched data

  print("Display Un Matched Data....")
  filtered_df.show(5, False)

# Display Un Matched Data....
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+--------------------------------------+----------------------------------------+----------+
# |StudentID|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|PlacementStatus|Extracurricular_indexed|PlacementTraining_indexed|features                                    |label|rawPrediction                         |probability                             |prediction|
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+--------------------------------------+----------------------------------------+----------+
# |18       |7.6 |1.0        |2.0     |1.0                     |71.0             |4.1             |Yes                      |Yes              |64.0     |71.0     |Placed         |0.0                    |0.0                      |[7.6,1.0,2.0,1.0,71.0,4.1,0.0,0.0,64.0,71.0]|1.0  |[16.0,4.0]                            |[0.8,0.2]                               |0.0       |
# |61       |7.9 |2.0        |2.0     |0.0                     |76.0             |4.3             |Yes                      |Yes              |87.0     |83.0     |NotPlaced      |0.0                    |0.0                      |[7.9,2.0,2.0,0.0,76.0,4.3,0.0,0.0,87.0,83.0]|0.0  |[7.0,13.0]                            |[0.35,0.65]                             |1.0       |
# |78       |8.3 |1.0        |1.0     |2.0                     |79.0             |4.0             |Yes                      |Yes              |84.0     |80.0     |NotPlaced      |0.0                    |0.0                      |[8.3,1.0,1.0,2.0,79.0,4.0,0.0,0.0,84.0,80.0]|0.0  |[7.944444444444445,12.055555555555555]|[0.39722222222222225,0.6027777777777777]|1.0       |
# |80       |7.8 |0.0        |2.0     |0.0                     |90.0             |4.7             |Yes                      |Yes              |68.0     |72.0     |NotPlaced      |0.0                    |0.0                      |[7.8,0.0,2.0,0.0,90.0,4.7,0.0,0.0,68.0,72.0]|0.0  |[3.0,17.0]                            |[0.15,0.85]                             |1.0       |
# |80       |8.6 |1.0        |1.0     |1.0                     |82.0             |4.6             |No                       |Yes              |63.0     |77.0     |Placed         |1.0                    |0.0                      |[8.6,1.0,1.0,1.0,82.0,4.6,1.0,0.0,63.0,77.0]|1.0  |[17.0,3.0]                            |[0.85,0.15]                             |0.0       |
# +---------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+---------------+-----------------------+-------------------------+--------------------------------------------+-----+--------------------------------------+----------------------------------------+----------+

######## Save your rf_model as 'Model_Campus_Place_GBT',
# since the GBT shows greater Accuracy level

  gbt_model.write().overwrite() \
    .save("/content/drive/MyDrive/Model_Campus_Place_GBT")

  print()
  display(HTML("<h3 style='color:magenta; text-align:left;'>Model Building Finished...</h3>"))

# Record the end time after the process has finished
  end_time = datetime.datetime.now()

# Calculate the time difference
  time_difference = end_time - current_time

# Print the time difference
  formatted_time_difference = str(time_difference).split('.')[0]  # Remove microseconds
  print("Time Difference :", formatted_time_difference)

  main_function()

def single_input_and_predict(event):

  print()
  clear_result_grid()

  filename = '/content/drive/MyDrive/Single_Campus_Place.csv'

  headers = ["Student_Name","CGPA","Internships","Projects","Workshops_Certifications",
             "AptitudeTestScore","SoftSkillsRating","ExtracurricularActivities",
             "PlacementTraining","SSC_Marks","HSC_Marks"
            ]
  css = """
      <style>
          .red-text .widget-label {
              color: red !important;
              font-size: 18px;
          }
      </style>
      """
# Define the input box or frame style

####### background-colors are

# "lightgray', "whitesmoke", "lavender", "mintcream", "aliceblue", "linen"
# "ivory", "honeydew", "floralwhite", "ghostwhite"


########### Define the CSS class with the desired style for frame

# In the code given below, the f-string is used to define the custom CSS (Cascading Style Sheets)
# for an input box. The variable input_box_style is expected to contain the CSS styling properties
# for the input box. The f-string allows the variable's value to be inserted directly into the string,
# resulting in a dynamic CSS style applied to the input box.


# The following display(HTML(css)), activate all css declarations

  display(HTML(css))

  display(HTML("<h2 style='color:green; text-align:center;'>Input your Student details</h2>"))

# Actual GUI descriptions for each required entry parameters declared here

  Student_Name = widgets.Text(description =
                              'Student Name>>>>>>>>>>>>>>',
                              style={'description_width': 'initial'},
                              layout=widgets.Layout(
                              width='450px',
                              border='2px solid blue',  # Blue, Add border style here
                              border_radius='5px',      # Add border-radius style here
                              padding='1px'))

#  Use Placeholder to assist data entry

  CGPA_ph = "Min = 6.0, Max = 9.9"
  CGPA  		= widgets.Text(description	=
                            'CGPA>>>>>>>>>>>>>>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=CGPA_ph)

  Internships_ph = "Min = 0, Max = 2"
  Internships		 	= widgets.Text(description	=
                            'No.of Internships>>>>>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=Internships_ph)

  Projects_ph = "Min = 0, Max = 3"
  Projects			 		= widgets.Text(description	=
                            'No.of Projects done>>>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=Projects_ph)

  Workshops_Certifications_ph = "Min = 0, Max = 3"
  Workshops_Certifications 	= widgets.Text(description	=
                            'No.of Workshops>>>>>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=Workshops_Certifications_ph)

  AptitudeTestScore_ph = "Min = 60, Max = 90"
  AptitudeTestScore		 	= widgets.Text(description	=
                            'Aptitude Test Score>>>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=AptitudeTestScore_ph)

  SoftSkillsRating_ph = "Min = 3.0, Max = 4.9"
  SoftSkillsRating		 	= widgets.Text(description	=
                            'Soft Skills Rating>>>>>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=SoftSkillsRating_ph)


# Define the options for the dropdown

  extra_curricular_options = ['Yes', 'No']

  ExtracurricularActivities = widgets.Dropdown(description	=
                            'Extra Curricular>>>>>>>>>>>>',
                            style={'description_width': 'initial'},
                            options=extra_curricular_options,
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'))

  PlacementTraining_options = ['Yes', 'No']

  PlacementTraining			 = widgets.Dropdown(description	=
                            'Placement Training>>>>>>>>>',
                            style={'description_width': 'initial'},
                            options=PlacementTraining_options,
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'))


  SSC_Marks_ph = "Min = 50.0, Max = 90.0"
  SSC_Marks				 	= widgets.Text(description	=
                            'SSC Marks Obtained>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=SSC_Marks_ph)

  HSC_Marks_ph = "Min = 50.0, Max = 90.0"
  HSC_Marks				 	= widgets.Text(description	=
                            'HSC Marks Obtained>>>>>>>',
                            style={'description_width': 'initial'},
                            layout=widgets.Layout(
                            width='400px',
                            border='2px solid blue',
                            border_radius='5px',
                            padding='1px'),
							              placeholder=HSC_Marks_ph)


# Add CSS classes declared above to the input widgets

  Student_Name.add_class('red-text')
  CGPA.add_class('red-text')
  Internships.add_class('red-text')
  Projects.add_class('red-text')
  Workshops_Certifications.add_class('red-text')
  AptitudeTestScore.add_class('red-text')
  SoftSkillsRating.add_class('red-text')
  ExtracurricularActivities.add_class('red-text')
  PlacementTraining.add_class('red-text')
  SSC_Marks.add_class('red-text')
  HSC_Marks.add_class('red-text')

# Display Data Entry headings and comments

  Student_Name_container = widgets.HBox(children=[Student_Name])
  CGPA_container = widgets.HBox(children=[CGPA])
  Internships_container = widgets.HBox(children=[Internships])
  Projects_container = widgets.HBox(children=[Projects])
  Workshops_Certifications_container = widgets.HBox(children=[Workshops_Certifications])
  AptitudeTestScore_container = widgets.HBox(children=[AptitudeTestScore])
  SoftSkillsRating_container = widgets.HBox(children=[SoftSkillsRating])
  ExtracurricularActivities_container = widgets.HBox(children=[ExtracurricularActivities])
  PlacementTraining_container = widgets.HBox(children=[PlacementTraining])
  SSC_Marks_container = widgets.HBox(children=[SSC_Marks])
  HSC_Marks_container = widgets.HBox(children=[HSC_Marks])

  display(Student_Name_container)
  display(CGPA_container)
  display(Internships_container)
  display(Projects_container)
  display(Workshops_Certifications_container)
  display(AptitudeTestScore_container)
  display(SoftSkillsRating_container)
  display(ExtracurricularActivities_container)
  display(PlacementTraining_container)
  display(SSC_Marks_container)
  display(HSC_Marks_container)

  print()

# Display Main Heading

  # display(HTML("<h2 style='color:green; text-align:center;'>Prediction using Single Input</h2>"))
  # print()
# Declare your buttons

  save_button_single = widgets.Button(description="Save Data")
  predict_button_single = widgets.Button(description="Predict Result")
  exit_button_single = widgets.Button(description="Go Back to Main Menu")

# Apply HTML/CSS for the above buttons

  custom_css_single = """
    <style>
    .custom-button-single {
        width: 250px;  /* Change this value for width */
        height: 50px;  /* Change this value for height */
        color: blue; /* Text color */
        background-color: yellow;  /* Background color */
        border: 2px solid black;
    }
  /* Add a mouse hover effect */

    .custom-button-single:hover {
        background-color: white; /* Change background color on mouse hover */
    }
    </style>
    """
# Attach the above style sheet to each buttons

  save_button_single.add_class("custom-button-single")
  predict_button_single.add_class("custom-button-single")
  exit_button_single.add_class("custom-button-single")

# Display the custom button

  display(HTML(custom_css_single))

  buttons_horizontal_layout_single = widgets.HBox([save_button_single,predict_button_single,
                                              exit_button_single])

# # Align the buttons using the following

  align_layout_single = widgets.Layout(justify_content='center')

# Arrange buttons horizontally, in center of the output area

  left_container_single = widgets.Box([buttons_horizontal_layout_single],
                                      layout=align_layout_single)

  display(left_container_single)
  print()

  def on_button_clicked(save_button_single):

# # Assign values from text boxes to variables

    entered_Student_Name = Student_Name.value
    entered_CGPA = CGPA.value
# Convert the value in Text Box to float as follows
# The red tilde under 'CGPA' is only a warning, not an error
    entered_CGPA = float(entered_CGPA)
    entered_Internships = Internships.value
    entered_Internships = float(entered_Internships)
    entered_Projects = Projects.value
    entered_Projects = float(entered_Projects)
    entered_Workshops_Certifications = Workshops_Certifications.value
    entered_Workshops_Certifications = float(entered_Workshops_Certifications)
    entered_AptitudeTestScore = AptitudeTestScore.value
    entered_AptitudeTestScore = float(entered_AptitudeTestScore)
    entered_SoftSkillsRating = SoftSkillsRating.value
    entered_SoftSkillsRating = float(entered_SoftSkillsRating)
    entered_ExtracurricularActivities = ExtracurricularActivities.value
    entered_PlacementTraining = PlacementTraining.value
    entered_SSC_Marks = SSC_Marks.value
    entered_SSC_Marks = float(entered_SSC_Marks)
    entered_HSC_Marks = HSC_Marks.value
    entered_HSC_Marks = float(entered_HSC_Marks)

# # Save your data as csv file 'user_input_house.csv'
    with open(filename, 'w', newline='') as f:
         writer = csv.writer(f)
         writer.writerow(headers)
         writer.writerow([entered_Student_Name,
                          entered_CGPA, entered_Internships,
                          entered_Projects,
                          entered_Workshops_Certifications,
                          entered_AptitudeTestScore,
                          entered_SoftSkillsRating,
                          entered_ExtracurricularActivities,
                          entered_PlacementTraining,
                          entered_SSC_Marks,
                          entered_HSC_Marks
                          ])
# Cross verify
         print('Data has been written to', filename)

  save_button_single.on_click(on_button_clicked)
  predict_button_single.on_click(predict_single)
  exit_button_single.on_click(go_back_function)


def predict_single(event):
  print("Control in single input predict...")
  clear_result_grid()
  print()
  display(HTML("<h3 style='color:orange; text-align:left;'>Single Input Data Prediction Started Please...wait !!!</h3>"))
# # Read Single Input Data for prediction

  single_df_from_csv = my_spark.read \
                  .format("csv") \
                  .option("header", "True") \
                  .option("inferSchema", "True") \
                  .load("/content/drive/MyDrive/Single_Campus_Place.csv")

  print("Schema of single_df_from_csv")
  single_df_from_csv.printSchema()
# Schema of single_df_from_csv
# root
#  |-- Student_Name: string (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: double (nullable = true)
#  |-- Projects: double (nullable = true)
#  |-- Workshops_Certifications: double (nullable = true)
#  |-- AptitudeTestScore: double (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: double (nullable = true)
#  |-- HSC_Marks: double (nullable = true)

  print("Data in single_df_from_csv")
  single_df_from_csv.show()

# Data in single_df_from_csv
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+
# |Student_Name|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+
# |   AMALDEV-P| 7.0|        2.0|     2.0|                     0.0|              7.0|             4.0|                      Yes|               No|     65.0|     69.0|
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+

# Since 'ExtracurricularActivities' and 'PlacementTraining' are string
# first convert it to decimal equivalent vector

  single_Extracurricular_converted = single_df_from_csv.withColumn(
                   "Extracurricular_indexed",
                   when(single_df_from_csv.ExtracurricularActivities == "Yes", 0.0)
                  .when(single_df_from_csv.ExtracurricularActivities == "No", 1.0)
                  .otherwise(2.0)
                          )

  single_PlaceTraining_converted = single_Extracurricular_converted.withColumn(
                   "PlacementTraining_indexed",
                   when(single_Extracurricular_converted.PlacementTraining == "Yes", 0.0)
                  .when(single_Extracurricular_converted.PlacementTraining == "No", 1.0)
                  .otherwise(2.0)
                          )

# Cross Verify
  print("Schema of single_PlaceTraining_converted")
  single_PlaceTraining_converted.printSchema()
# Schema of single_PlaceTraining_converted
# root
#  |-- Student_Name: string (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: double (nullable = true)
#  |-- Projects: double (nullable = true)
#  |-- Workshops_Certifications: double (nullable = true)
#  |-- AptitudeTestScore: double (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: double (nullable = true)
#  |-- HSC_Marks: double (nullable = true)
#  |-- Extracurricular_indexed: double (nullable = false)
#  |-- PlacementTraining_indexed: double (nullable = false)

# Select your feature columns for prediction

  single_feature_cols = ["CGPA","Internships","Projects","Workshops_Certifications",
                      "AptitudeTestScore","SoftSkillsRating","Extracurricular_indexed",
                      "PlacementTraining_indexed","SSC_Marks","HSC_Marks"
                        ]
  single_output_col = "features"


# Vectorize the above feature_cols and add it as a list in single_df_gender_converted

  single_assembler = VectorAssembler(inputCols=single_feature_cols,
                                     outputCol=single_output_col)

  single_df_final_vectors = single_assembler.transform(single_PlaceTraining_converted)

# Cross Verify features data

  print("Display few Data in single_df_final_vectors")
  single_df_final_vectors.show()
# Display few Data in single_df_final_vectors
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+-------------------------------------------+
# |Student_Name|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|Extracurricular_indexed|PlacementTraining_indexed|features                                   |
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+-------------------------------------------+
# |AMALDEV-P   |7.0 |2.0        |2.0     |0.0                     |7.0              |4.0             |Yes                      |No               |65.0     |69.0     |0.0                    |1.0                      |[7.0,2.0,2.0,0.0,7.0,4.0,0.0,1.0,65.0,69.0]|
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+-------------------------------------------+

# Load your saved model from "/content/drive/MyDrive/Model_Campus_Place_GBT"

  loaded_model = GBTClassificationModel \
        .load("/content/drive/MyDrive/Model_Campus_Place_GBT")

# predict Selected / NotSelected

  single_input_prediction = loaded_model.transform(single_df_final_vectors)

  print("Prediction Output using Single user input...")
  single_input_prediction.show()
# Prediction Output using Single user input...
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------+--------------------+--------------------+----------+
# |Student_Name|CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|Extracurricular_indexed|PlacementTraining_indexed|            features|       rawPrediction|         probability|prediction|
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------+--------------------+--------------------+----------+
# |   AMALDEV-P| 7.0|        2.0|     2.0|                     0.0|              7.0|             4.0|                      Yes|               No|     65.0|     69.0|                    0.0|                      1.0|[7.0,2.0,2.0,0.0,...|[-0.0125397442583...|[0.49373045648587...|       1.0|
# +------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------+--------------------+--------------------+----------+

# # Display Prediction Result as 'Selected / Not Selected'

  selected_df = single_input_prediction.withColumn("Final_Output",
                  when(single_input_prediction["prediction"] == 1, "Placed")
                  .otherwise("Not Placed"))

  selected_df_final = selected_df.select("Student_Name","Final_Output")

########## Display prediction results in color frame

# Convert final_prediction to Pandas DataFrame

  pandas_df = selected_df_final.toPandas()

# Apply color to the DataFrame output

# The applymap method is used to apply a function to each element of the DataFrame
# the lambda function in this code snippet is used to assign the CSS style 'color: red'
# to the cells in the 'Price' column of the DataFrame when generating its styled representation.

# background colors, 'red','green','blue','yellow','orange','purple','pink'

  styled_df = pandas_df.style.applymap(lambda x: 'background-color: aliceblue',
                                       subset=["Student_Name","Final_Output"])

# Generate an HTML table(html_table) with customized CSS
# 'th' = Table Heading, 'td' = Table Data

  html_table = styled_df.hide(axis="index").set_table_styles(
        [{'selector': 'th', 'props':[('color', 'red'), ('padding', '5px')]},
         {'selector': 'td', 'props': [('vertical-align', 'middle'),('padding', '5px')]}
        ]).to_html()

# Draw a frame around your output and Display the contents

# the f prefix enables you to embed expressions within a string by enclosing them in curly braces {}
# Within the string, you have embedded an expression {html_table}.
# The value of the html_table variable is dynamically inserted into the string at that location, that
# values are nothing but your prediction results with headings.

  output_frame = f"""
      <div style="border: 2px solid black; padding: 10px;width: 50%; background-color: lightgrey;">
          <h3 style='color:blue; text-align:left;'>Final Prediction as per your Input</h3>
          {html_table}
      </div>
      """

  display(HTML(output_frame))

####### Final Output

# Final Prediction as per your Input
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Student_Name	Final_Output
#---------------------------
# AMALDEV-P	      Placed
######## Single User Data Input and Prediction ends here

# Go Back to Sub Menu

  button = widgets.Button(description="Click Here to Go Back")
  button.on_click(on_button_click_goback)

  # Display the button
  display(widgets.HBox([button]))


def on_button_click_goback(event):
  single_input_and_predict(event)

def bulk_data_load(event):
  print()
  clear_result_grid()
  set_focus_cell2()

# Declare your upload buttons

  browse_button = widgets.Button(description="Click this and select Choose Files")
  predict_button = widgets.Button(description="Predict Result")
  exit_button = widgets.Button(description="Go Back to Main Menu")

# Apply HTML/CSS for the above buttons

  custom_css_bulk = """
    <style>
    .custom-button-bulk {
        width: 250px;  /* Change this value for width */
        height: 50px;  /* Change this value for height */
        color: blue; /* Text color */
        background-color: yellow;  /* Background color */
        border: 2px solid black;
    }
  /* Add a mouse hover effect */

    .custom-button-bulk:hover {
        background-color: white; /* Change background color on mouse hover */
    }
    </style>
    """
# Attach the above style sheet to each buttons


  browse_button.add_class("custom-button-bulk")
  predict_button.add_class("custom-button-bulk")
  exit_button.add_class("custom-button-bulk")

# Display the custom button

  display(HTML(custom_css_bulk))

  buttons_horizontal_layout_1 = widgets.HBox([browse_button,predict_button,exit_button])

# # Align the buttons using the following

  align_layout = widgets.Layout(justify_content='center')

# Arrange buttons horizontally, in center of the output area

  left_container_1 = widgets.Box([buttons_horizontal_layout_1], layout=align_layout)

  print()
  print()
  print()
  print()
  display(HTML("<h3 style='color:magenta; text-align:center;'>Bulk Input Data Prediction Sub Menu</h3>"))
  print()
  display(left_container_1)
  print()
  print()
  print()
  print()

# The !mv command is used to copy dataset from 'sample_data' folder inside
# google drive to /content/drive/MyDive. Other wise after 48 hrs
# the data in sample_data will be removed.

# The dollar sign $ is used in the ! shell command execution within
# Google Colab to indicate that you're referencing a variable within
# a string. When you use $variable_name within a shell command
# executed in a code cell, it is replaced with the value of the variable.

# The 'selected_file' after 'files.upload' will contain a key,value pair
# To extract the file name, for loop is
# used to get the key, in other words actual file name.

  def upload_file_fn(event):
      selected_file = files.upload()
      for dataset_name in selected_file.keys():
            source_path = f"/content/{dataset_name}"
            destination_path = f"/content/drive/MyDrive/{dataset_name}"
            !mv "$source_path" "$destination_path"

  browse_button.on_click(upload_file_fn)
  predict_button.on_click(bulk_predict_fn)
  exit_button.on_click(go_back_function)

# 'exit_function' is used separately because you cannot call 'main_function()' directly
# in on_click event. '0 argument expected but 1 is given' error will be flagged

def bulk_predict_fn(event):

# Read Bulk Input Data for prediction

  clear_result_grid()
  print()
  display(HTML("<h3 style='color:orange; text-align:left;'>Bulk Input Data Prediction Started Please...wait !!!</h3>"))

  bulk_df_from_csv = my_spark.read \
                  .format("csv") \
                  .option("header", "True") \
                  .option("inferSchema", "True") \
                  .load("/content/drive/MyDrive/Bulk_Campus_Place.csv")

  # print("Schema of bulk_df_from_CSV")
  # bulk_df_from_csv.printSchema()

# Schema of bulk_df_from_CSV
# root
#  |-- Student_Name: string (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: integer (nullable = true)
#  |-- Projects: integer (nullable = true)
#  |-- Workshops_Certifications: integer (nullable = true)
#  |-- AptitudeTestScore: integer (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: integer (nullable = true)
#  |-- HSC_Marks: integer (nullable = true)

  # print("Data in bulk_df_from_csv")
  # bulk_df_from_csv.show(5, False)

# Data in bulk_df_from
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+
# |Student_Name    |CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+
# |AYUSH ANTONY-np |6.9 |0          |1       |0                       |66               |4.4             |Yes                      |No               |56       |67       |
# |BHAGYA BAIJU-p  |6.5 |2          |3       |3                       |89               |4.8             |Yes                      |No               |86       |84       |
# |DIVINE JOSE-p   |8.2 |2          |1       |2                       |90               |4.8             |Yes                      |Yes              |83       |81       |
# |DONALD ANTONY-np|6.6 |1          |2       |1                       |78               |4.3             |No                       |Yes              |58       |70       |
# |EDWIN FELIX-np  |6.8 |1          |3       |0                       |73               |4.3             |No                       |Yes              |59       |73       |
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+


  bulk_Extracurricular_converted = bulk_df_from_csv.withColumn(
                   "Extracurricular_indexed",
                   when(bulk_df_from_csv.ExtracurricularActivities == "Yes", 0.0)
                  .when(bulk_df_from_csv.ExtracurricularActivities == "No", 1.0)
                  .otherwise(2.0)
                          )

  bulk_PlaceTraining_converted = bulk_Extracurricular_converted.withColumn(
                   "PlacementTraining_indexed",
                   when(bulk_Extracurricular_converted.PlacementTraining == "Yes", 0.0)
                  .when(bulk_Extracurricular_converted.PlacementTraining == "No", 1.0)
                  .otherwise(2.0)
                          )

# Cross Verify

  # print("Schema of bulk_PlaceTraining_converted")
  # bulk_PlaceTraining_converted.printSchema()

# Schema of bulk_PlaceTraining_converted
# root
#  |-- Student_Name: string (nullable = true)
#  |-- CGPA: double (nullable = true)
#  |-- Internships: integer (nullable = true)
#  |-- Projects: integer (nullable = true)
#  |-- Workshops_Certifications: integer (nullable = true)
#  |-- AptitudeTestScore: integer (nullable = true)
#  |-- SoftSkillsRating: double (nullable = true)
#  |-- ExtracurricularActivities: string (nullable = true)
#  |-- PlacementTraining: string (nullable = true)
#  |-- SSC_Marks: integer (nullable = true)
#  |-- HSC_Marks: integer (nullable = true)
#  |-- Extracurricular_indexed: double (nullable = false)
#  |-- PlacementTraining_indexed: double (nullable = false)

# Select your feature columns for prediction

  bulk_feature_cols = ["CGPA","Internships","Projects","Workshops_Certifications",
                      "AptitudeTestScore","SoftSkillsRating","Extracurricular_indexed",
                      "PlacementTraining_indexed","SSC_Marks","HSC_Marks"
                        ]
  bulk_output_col = "features"

# Vectorize the above feature_cols and add it as a list in df_final_transformed

  bulk_assembler = VectorAssembler(inputCols=bulk_feature_cols, outputCol=bulk_output_col)

  bulk_df_final_vectors = bulk_assembler.transform(bulk_PlaceTraining_converted)

# Cross Verify features data

  # print("Display few Data in bulk_df_final_vectors")
  # bulk_df_final_vectors.show(5, False)

# Display few Data in bulk_df_final_vectors
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------------------------------+
# |Student_Name    |CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|Extracurricular_indexed|PlacementTraining_indexed|features                                    |
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------------------------------+
# |AYUSH ANTONY-np |6.9 |0          |1       |0                       |66               |4.4             |Yes                      |No               |56       |67       |0.0                    |1.0                      |[6.9,0.0,1.0,0.0,66.0,4.4,0.0,1.0,56.0,67.0]|
# |BHAGYA BAIJU-p  |6.5 |2          |3       |3                       |89               |4.8             |Yes                      |No               |86       |84       |0.0                    |1.0                      |[6.5,2.0,3.0,3.0,89.0,4.8,0.0,1.0,86.0,84.0]|
# |DIVINE JOSE-p   |8.2 |2          |1       |2                       |90               |4.8             |Yes                      |Yes              |83       |81       |0.0                    |0.0                      |[8.2,2.0,1.0,2.0,90.0,4.8,0.0,0.0,83.0,81.0]|
# |DONALD ANTONY-np|6.6 |1          |2       |1                       |78               |4.3             |No                       |Yes              |58       |70       |1.0                    |0.0                      |[6.6,1.0,2.0,1.0,78.0,4.3,1.0,0.0,58.0,70.0]|
# |EDWIN FELIX-np  |6.8 |1          |3       |0                       |73               |4.3             |No                       |Yes              |59       |73       |1.0                    |0.0                      |[6.8,1.0,3.0,0.0,73.0,4.3,1.0,0.0,59.0,73.0]|
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------------------------------+

# Load your saved GBT model from "/content/drive/MyDrive/Model_Campus_Place_GBT"

  loaded_model = GBTClassificationModel \
        .load("/content/drive/MyDrive/Model_Campus_Place_GBT")

# predict alzheimer detected or not, prediction = 1 No Alzheimer,prediction = 0 Alzheimer Detected

  bulk_input_prediction = loaded_model.transform(bulk_df_final_vectors)

  # print("Prediction Output using Bulk user input...")
  # bulk_input_prediction.show(5, False)

# Prediction Output using Bulk user input...
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------------------------------+----------------------------------------+-----------------------------------------+----------+
# |Student_Name    |CGPA|Internships|Projects|Workshops_Certifications|AptitudeTestScore|SoftSkillsRating|ExtracurricularActivities|PlacementTraining|SSC_Marks|HSC_Marks|Extracurricular_indexed|PlacementTraining_indexed|features                                    |rawPrediction                           |probability                              |prediction|
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------------------------------+----------------------------------------+-----------------------------------------+----------+
# |AYUSH ANTONY-np |6.9 |0          |1       |0                       |66               |4.4             |Yes                      |No               |56       |67       |0.0                    |1.0                      |[6.9,0.0,1.0,0.0,66.0,4.4,0.0,1.0,56.0,67.0]|[1.3345658180032591,-1.3345658180032591]|[0.9351804127850412,0.06481958721495884] |0.0       |
# |BHAGYA BAIJU-p  |6.5 |2          |3       |3                       |89               |4.8             |Yes                      |No               |86       |84       |0.0                    |1.0                      |[6.5,2.0,3.0,3.0,89.0,4.8,0.0,1.0,86.0,84.0]|[-1.3971560375367806,1.3971560375367806]|[0.057632315675267436,0.9423676843247326]|1.0       |
# |DIVINE JOSE-p   |8.2 |2          |1       |2                       |90               |4.8             |Yes                      |Yes              |83       |81       |0.0                    |0.0                      |[8.2,2.0,1.0,2.0,90.0,4.8,0.0,0.0,83.0,81.0]|[-1.3888810362957775,1.3888810362957775]|[0.058537768398425095,0.9414622316015749]|1.0       |
# |DONALD ANTONY-np|6.6 |1          |2       |1                       |78               |4.3             |No                       |Yes              |58       |70       |1.0                    |0.0                      |[6.6,1.0,2.0,1.0,78.0,4.3,1.0,0.0,58.0,70.0]|[0.8283113200780285,-0.8283113200780285]|[0.8397841115454632,0.16021588845453683] |0.0       |
# |EDWIN FELIX-np  |6.8 |1          |3       |0                       |73               |4.3             |No                       |Yes              |59       |73       |1.0                    |0.0                      |[6.8,1.0,3.0,0.0,73.0,4.3,1.0,0.0,59.0,73.0]|[1.3798380832136206,-1.3798380832136206]|[0.9404575028402804,0.0595424971597196]  |0.0       |
# +----------------+----+-----------+--------+------------------------+-----------------+----------------+-------------------------+-----------------+---------+---------+-----------------------+-------------------------+--------------------------------------------+----------------------------------------+-----------------------------------------+----------+

# Extract selected columns from bulk_input_prediction

  selected_df = bulk_input_prediction.select("Student_Name","prediction")

# Cross Verify

  # print("Display Selected columns from selected_df...")
  # selected_df.show(5, False)

# Display Selected columns from selected_df...
# +----------------+----------+
# |Student_Name    |prediction|
# +----------------+----------+
# |AYUSH ANTONY-np |0.0       |
# |BHAGYA BAIJU-p  |1.0       |
# |DIVINE JOSE-p   |1.0       |
# |DONALD ANTONY-np|0.0       |
# |EDWIN FELIX-np  |0.0       |
# +----------------+----------+

# Add the "Final_Output" column based on the condition

  selected_df = bulk_input_prediction.withColumn("Final_Output",
                  when(bulk_input_prediction["prediction"] == 1, "Placed")
                  .otherwise("Not Placed"))

  selected_df_final = selected_df.select("Student_Name","Final_Output")

########## Display prediction results in color frame

# Convert final_prediction to Pandas DataFrame

  pandas_df = selected_df_final.toPandas()

# # Display the top 5 rows

#   print(pandas_df.head())

# Apply color to the DataFrame output

# The applymap method is used to apply a function to each element of the DataFrame
# the lambda function in this code snippet is used to assign the CSS style 'color: red'
# to the cells in the 'Price' column of the DataFrame when generating its styled representation.

# background colors, 'red','green','blue','yellow','orange','purple','pink'

  styled_df = pandas_df.style.applymap(lambda x: 'background-color: aliceblue',
                                       subset=["Student_Name","Final_Output"])

# Generate an HTML table(html_table) with customized CSS
# 'th' = Table Heading, 'td' = Table Data

  html_table = styled_df.hide(axis="index").set_table_styles(
        [{'selector': 'th', 'props':[('color', 'red'), ('padding', '5px')]},
         {'selector': 'td', 'props': [('vertical-align', 'middle'),('padding', '5px')]}
        ]).to_html()

# Draw a frame around your output and Display the contents

# the f prefix enables you to embed expressions within a string by enclosing them in curly braces {}
# Within the string, you have embedded an expression {html_table}.
# The value of the html_table variable is dynamically inserted into the string at that location, that
# values are nothing but your prediction results with headings.

  output_frame = f"""
      <div style="border: 2px solid black; padding: 10px;width: 50%; background-color: lightgrey;">
          <h3 style='color:blue; text-align:left;'>Final Prediction as per your Input</h3>
          {html_table}
      </div>
      """

  display(HTML(output_frame))

# Final Prediction as per your Input
# ----------------------------------
# Student_Name	        Final_Output
# -----------------------------------
# AYUSH ANTONY-np	        Not Placed
# BHAGYA BAIJU-p	        Placed
# DIVINE JOSE-p	          Placed
# DONALD ANTONY-np	      Not Placed
# EDWIN FELIX-np	        Not Placed
# FEMI JERRINA-p	        Placed
# GOKUL V.S.-p	          Placed
# GRESHMA P.J-np	        Not Placed
# JEENA CRUZ-np	          Not Placed
# JISS GEORGEKUTTY-np	    Not Placed
# JOSE JOY-np	Not         Placed
# JOSNA JOSEPH-p	        Placed
# MICHAEL SHINOY-p	      Placed
# NAVANEETH KRISHNAN-np	  Not Placed
# NIVYA BABU-p	          Placed
# RAPHEAL. V-np	          Not Placed
# RESHMA MICHAEL-np	Not   Placed
# RESHMA RODRIGUES-p	    Placed
# RICHARD THOMAS-p	      Placed
# SAHITHYA-p	            Placed

########## Bulk prediction ends here
  print()
  display(HTML("<h3 style='color:orange; text-align:left;'>Bulk Input Data Prediction Finished !!!</h3>"))
  main_function()

def go_back_function(event):
  clear_result_grid()
  main_function()

  print()

def main_function():

#  clear_result_grid()
# Set focus to result grid
  set_focus_cell2()
  print()
  print()
  print()
  print()
  display(HTML("<h3 style='color:magenta; text-align:center;'>Main Menu - Campus Placemnt Prediction</h3>"))
  print()

# Create your buttons for each events

  load_data_button = widgets.Button(description="1. Load Data")
  eda_analysis_button = widgets.Button(description="2. EDA Analysis")
  pre_process_button = widgets.Button(description="3. Pre Process Data")
  transform_data_button = widgets.Button(description="4. Transform Data")
  build_model_and_save_button = widgets.Button(description="5. Build Model & Save")
  single_mail_predict_button = widgets.Button(description="6. Prediction(Single Input)")
  bulk_mail_predict_button = widgets.Button(description="7. Prediction(Bulk Input)")

# Assign python functions as for each Menu events

  load_data_button.on_click(load_data)
  eda_analysis_button.on_click(EDA_start)
  pre_process_button.on_click(pre_process_data)
  transform_data_button.on_click(transformation)
  build_model_and_save_button.on_click(build_model_save)
  single_mail_predict_button.on_click(single_input_and_predict)
  bulk_mail_predict_button.on_click(bulk_data_load)

# Define custom CSS to adjust button dimensions, backcolor and textcolor

# Here is a list of some popular CSS colors:

# red,green,blue,yellow,black,white,gray,purple,orange,pink,brown,cyan,magenta

# You can also use more specific colors using HEXADECIMAL coded, such as:
# For more hexa color codes go to " https://www.color-hex.com/"

# #FF0000 (bright red), #00FF00 (bright green), #0000FF (bright blue), #FFFFFF (pure white)
# #000000 (pure black). #808080 (gray)

########################
# For more details regarding Background color and foreground text go to
# https://www.uxmatters.com/mt/archives/2007/01/applying-color-theory-to-digital-displays.php
########################

  custom_css = """
  <style>
  .custom-button {
      width: 200px;  /* Change this value for width */
      height: 50px;  /* Change this value for height */
      color: blue; /* Text color */
      background-color: lightgreen;  /* Background color */
      border: 2px solid black;
  }

/* Add a mouse hover effect */

  .custom-button:hover {
      background-color: white; /* Change background color on mouse hover */
      }
  </style>

  """

  # Apply CSS class properties to buttons

  load_data_button.add_class("custom-button")
  eda_analysis_button.add_class("custom-button")
  pre_process_button.add_class("custom-button")
  transform_data_button.add_class("custom-button")
  build_model_and_save_button.add_class("custom-button")
  single_mail_predict_button.add_class("custom-button")
  bulk_mail_predict_button.add_class("custom-button")

# Display the custom CSS and buttons
# ensuring that the CSS rules are applied to the button's appearance when
# they are displayed in the notebook.

  display(HTML(custom_css))

# Arrange buttons horizontally, HBox is used to display buttons horizontally
#                               VBox is used to display buttons vertically

  buttons_horizontal_layout_1 = widgets.HBox([load_data_button,
                                              eda_analysis_button, pre_process_button])
  buttons_horizontal_layout_2 = widgets.HBox([transform_data_button,
                                              build_model_and_save_button,
                                              single_mail_predict_button])
  buttons_horizontal_layout_3 = widgets.HBox([bulk_mail_predict_button])

# Center the buttons using a centered container in result grid

  centered_layout = widgets.Layout(justify_content='center')
  centered_container_2 = widgets.Box([buttons_horizontal_layout_2], layout=centered_layout)

# Arrange buttons horizontally, in center of the output area

  centered_container_1 = widgets.Box([buttons_horizontal_layout_1], layout=centered_layout)
  display(centered_container_1)
  centered_container_3 = widgets.Box([buttons_horizontal_layout_3], layout=centered_layout)

  # Display the centered buttons with one line gap between each row

  print()
  display(centered_container_2)
  print()
  display(centered_container_3)
  print()


# Declare a main function as follows to invoke all other function

if __name__ == "__main__":

# Call main function

  main_function()





Box(children=(HBox(children=(Button(description='Save Data', style=ButtonStyle(), _dom_classes=('custom-button…